In [1]:
import numpy as np
import torch
from PIL import Image
import cv2
import os
from tqdm import tqdm
import torchvision.transforms as T

/home/huangzhitong/anaconda3/envs/unicolor/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original_path = '../results/old_coco_259999/original'

test_paths = [
    '../results/old_coco_259999/colorized',
    '../results/new_coco_299999/colorized'
]

# FID score

In [3]:
from pytorch_fid import fid_score

for path in test_paths:
    fid = fid_score.calculate_fid_given_paths(paths=[original_path, path], batch_size=50, dims=2048, device='cuda:0')
    print(original_path, '\n', path, '\n', f'FID score: {fid}')

100%|██████████| 100/100 [01:06<00:00,  1.51it/s]


../results/old_coco_259999/original 
 ../results/old_coco_259999/colorized 
 FID score: 7.809005101211142


100%|██████████| 100/100 [01:07<00:00,  1.47it/s]


../results/old_coco_259999/original 
 ../results/new_coco_299999/colorized 
 FID score: 8.265271639647949


# Colorfulness

In [ ]:
def image_colorfulness(image):
	# split the image into its respective RGB components
	(B, G, R) = cv2.split(image.astype("float"))

	# compute rg = R - G
	rg = np.absolute(R - G)

	# compute yb = 0.5 * (R + G) - B
	yb = np.absolute(0.5 * (R + G) - B)

	# compute the mean and standard deviation of both `rg` and `yb`
	(rbMean, rbStd) = (np.mean(rg), np.std(rg))
	(ybMean, ybStd) = (np.mean(yb), np.std(yb))

	# combine the mean and standard deviations
	stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
	meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))

	# derive the "colorfulness" metric and return it
	return stdRoot + (0.3 * meanRoot)

for path in test_paths:
    files = os.listdir(path)
    colorful = []
    for i in tqdm(range(len(files))):
        img = Image.open( os.path.join(path, files[i]) ).convert('RGB').resize([256, 256])
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        colorful.append( image_colorfulness(img) )

    colorful = np.array(colorful)
    colorful = np.mean(colorful)
    print(path, '\n', f'Colorfulness: {colorful}')

## PSNR

In [ ]:
def preprocess(img, size):
    if size == None:
        size = [(img.size[1] // 16) * 16, (img.size[0] // 16) * 16]
    transform = T.Compose([T.Resize(size), T.ToTensor()])
    x = transform(img)
    x = x.unsqueeze(0)
    return x

with torch.no_grad():
    for path in test_paths:
        src_files = os.listdir(original_path)
        tgt_files = os.listdir(path)
        psnrs = []
        for i in tqdm(range(len(src_files))):
            assert src_files[i].split('.')[0] == tgt_files[i].split('.')[0]
            img0 = Image.open( os.path.join(original_path, src_files[i]) ).convert('RGB')
            img1 = Image.open( os.path.join(path, tgt_files[i]) ).convert('RGB')
            x0 = preprocess(img0, [256, 256])
            x1 = preprocess(img1, [256, 256])
            mse = torch.mean((x0 - x1) ** 2)
            psnr = 10 * torch.log10(1.0 / mse)
            psnrs.append(psnr.item())
        psnrs = np.mean(psnrs)
        print(original_path, '\n', path, '\n', f'PSNR: {psnrs}')

# LPIPS

In [ ]:
import lpips

def preprocess(img, size):
    if size == None:
        size = [(img.size[1] // 16) * 16, (img.size[0] // 16) * 16]
    transform = T.Compose([T.Resize(size), T.ToTensor()])
    x = transform(img)
    x = x * 2 - 1
    x = x.unsqueeze(0)
    return x

loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
#loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

with torch.no_grad():
    for path in test_paths:
        src_files = os.listdir(original_path)
        tgt_files = os.listdir(path)
        distance = []
        for i in tqdm(range(len(src_files))):
            assert src_files[i].split('.')[0] == tgt_files[i].split('.')[0]
            img0 = Image.open( os.path.join(original_path, src_files[i]) ).convert('RGB')
            img1 = Image.open( os.path.join(path, tgt_files[i]) ).convert('RGB')
            x0 = preprocess(img0, [256, 256])
            x1 = preprocess(img1, [256, 256])
            d = loss_fn_alex(x0, x1)
            distance.append(d)
        distance = torch.cat(distance, axis=0)
        distance = distance.mean()
        print(original_path, '\n', path, '\n', f'LPIPS: {distance}')

# SSIM

In [ ]:
from pytorch_msssim import ssim, SSIM

def preprocess(img, size):
    if size == None:
        size = [(img.size[1] // 16) * 16, (img.size[0] // 16) * 16]
    transform = T.Compose([T.Resize(size), T.ToTensor()])
    x = transform(img)
    x = x.unsqueeze(0)
    return x

ssim_module = SSIM(data_range=1, size_average=True, channel=3)
batch_size = 100

for path in test_paths:
    src_files = os.listdir(original_path)
    tgt_files = os.listdir(path)
    ssim_scores = []
    x0 = []
    x1 = []
    with torch.no_grad():
        for i in tqdm(range(len(src_files))):
            assert src_files[i].split('.')[0] == tgt_files[i].split('.')[0]
            img0 = Image.open( os.path.join(original_path, src_files[i]) ).convert('RGB')
            img1 = Image.open( os.path.join(path, tgt_files[i]) ).convert('RGB')
            x0.append(preprocess(img0, [256, 256]))
            x1.append(preprocess(img1, [256, 256]))

            if (i+1) % batch_size == 0:
                x0 = torch.cat(x0, dim=0)
                x1 = torch.cat(x1, dim=0)
                ssim_loss = ssim_module(x0, x1)
                ssim_scores.append(ssim_loss.cpu().numpy())
                x0 = []
                x1 = []
    ssim = np.mean(ssim_scores)
    print(original_path, '\n', path, '\n', f'SSIM: {ssim}')

    

# Contextual loss

In [ ]:
import contextual_loss as cl

def preprocess(img, size):
    if size == None:
        size = [(img.size[1] // 16) * 16, (img.size[0] // 16) * 16]
    transform = T.Compose([T.Resize(size), T.ToTensor()])
    x = transform(img)
    x = x.unsqueeze(0)
    return x

criterion = cl.ContextualLoss(use_vgg=False, loss_type='l1').cuda()

for path in test_paths:
    src_files = os.listdir(original_path)
    tgt_files = os.listdir(path)
    losses = []
    with torch.no_grad():
        for i in tqdm(range(len(src_files))):
            assert src_files[i].split('.')[0] == tgt_files[i].split('.')[0]
            img0 = Image.open( os.path.join(original_path, src_files[i]) ).convert('RGB')
            img1 = Image.open( os.path.join(path, tgt_files[i]) ).convert('RGB')
            x0 = preprocess(img0, [96, 96]).cuda()
            x1 = preprocess(img1, [96, 96]).cuda()

            loss = criterion(x0, x1)

            losses.append(loss.cpu().numpy())
            
    print(original_path, '\n', path, '\n', f'Contextual loss: {np.mean(losses)}')



In [ ]:
import contextual_loss as cl

def preprocess(img, size):
    if size == None:
        size = [(img.size[1] // 16) * 16, (img.size[0] // 16) * 16]
    transform = T.Compose([T.Resize(size), T.ToTensor()])
    x = transform(img)
    x = x.unsqueeze(0)
    return x

criterion = cl.ContextualLoss(use_vgg=False, loss_type='l2').cuda()

for path in test_paths:
    src_files = os.listdir(original_path)
    tgt_files = os.listdir(path)
    losses = []
    with torch.no_grad():
        for i in tqdm(range(len(src_files))):
            assert src_files[i].split('.')[0] == tgt_files[i].split('.')[0]
            img0 = Image.open( os.path.join(original_path, src_files[i]) ).convert('RGB')
            img1 = Image.open( os.path.join(path, tgt_files[i]) ).convert('RGB')
            x0 = preprocess(img0, [96, 96]).cuda()
            x1 = preprocess(img1, [96, 96]).cuda()

            loss = criterion(x0, x1)

            losses.append(loss.cpu().numpy())
            
    print(original_path, '\n', path, '\n', f'Contextual loss: {np.mean(losses)}')

In [ ]:
import contextual_loss as cl

def preprocess(img, size):
    if size == None:
        size = [(img.size[1] // 16) * 16, (img.size[0] // 16) * 16]
    transform = T.Compose([T.Resize(size), T.ToTensor()])
    x = transform(img)
    x = x.unsqueeze(0)
    return x

criterion = cl.ContextualLoss(use_vgg=False, loss_type='cosine').cuda()

for path in test_paths:
    src_files = os.listdir(original_path)
    tgt_files = os.listdir(path)
    losses = []
    with torch.no_grad():
        for i in tqdm(range(len(src_files))):
            assert src_files[i].split('.')[0] == tgt_files[i].split('.')[0]
            img0 = Image.open( os.path.join(original_path, src_files[i]) ).convert('RGB')
            img1 = Image.open( os.path.join(path, tgt_files[i]) ).convert('RGB')
            x0 = preprocess(img0, [96, 96]).cuda()
            x1 = preprocess(img1, [96, 96]).cuda()

            loss = criterion(x0, x1)

            losses.append(loss.cpu().numpy())
            
    print(original_path, '\n', path, '\n', f'Contextual loss: {np.mean(losses)}')